In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:93: MissingIDFieldWarning: Code cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.29/01


In [2]:
iOption0 = 0
is_sigmalised = 1

In [3]:
isigma = 2
N_files = 1
N_centr = 8
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_e = 1
centralities = ["1-2 layer pos","1-2 layer neg","2-3 layer pos","2-3 layer neg","2-4 layer pos","2-4 layer neg","3-4 layer pos","3-4 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5,6,6]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: layer_bins = [46,46,48,48,50,50,51,51,53,53,55,55]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-26th_QA_19198_350runs" #my-25th_QA_19197_1000runs
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [4]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/rg/my-26th_QA_19198_350runs0.root


In [5]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [6]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]

In [7]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.001*layer_bins[2*ilayer]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[icentr][1]-boards_fit2[icentr][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        if iOption0<4:
            Mean_local.append(total.GetParameter(1))
            Sigma_local.append(total.GetParameter(2))
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [8]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+is_sigmalised*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [9]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.2.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.3.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.5.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.6.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.6.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.8.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.9.png" for writing. Please check permissions.
python ERROR: cannot open image file "output/dynamic_delta_fits/6/0dynamic_0.9.png" for writing. Please check permissions.
python ERROR: ca

In [10]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [11]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-5.5+0.5,-4.4-0.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            else : 
                fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
                fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [12]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

array('d', [0.8561854012356819, 0.8693926761089948, 0.8925499032086678, 0.9238944646907058, 0.9623564331105485, 0.987738108637534, 0.9937781487832884, 0.990116002539102, 0.9921543438515034, 1.0046316728761424, 1.3393717059424464, 1.4846823788972918])

 array('d', [0.8669244318531399, 0.9453786992589173, 0.8314690566999782, 0.8273299227189609, 0.8255077500656705, 0.8380845719343364, 0.8558290597053755, 0.8750500439750173, 0.903099154503756, 0.9052394164323694, 0.8703236516091067, 0.8258064473082088])


In [13]:
print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][8][6][3]={
{
{
{0.942409, 0.0, -0.03575},
{0.940871, 0.0, -1.971993},
{0.926878, 0.0, -0.62573},
{0.120657, 0.80054, -0.01221},
{0.0, 0.943855, -0.042304},
{0.843754, 0.3336, -1.706907},
},
{
{0.802841, 0.260831, -1.562423},
{0.097078, 0.772161, -0.01912},
{0.868521, 0.0, -1.680968},
{0.877256, 0.0, -0.749224},
{0.001928, 0.883292, 0.028336},
{0.875117, 0.000636, 5.520022},
},
{
{0.881708, 6e-06, 10.229994},
{0.896473, 0.0, -0.119554},
{0.875149, 0.0, -1.291415},
{0.85981, 0.0, -1.406407},
{0.842013, 0.0, -1.698336},
{0.841925, 0.0, -1.152765},
},
{
{0.694433, 0.168784, -0.004903},
{0.85277, 0.0, -1.124947},
{0.870196, 0.0, -0.312914},
{0.026265, 0.862931, -0.012105},
{1e-06, 0.91849, -0.035313},
{0.608381, 0.017828, 10.229999},
},
{
{0.921545, 0.0, -0.753623},
{0.884184, 5e-06, 10.229993},
{0.892782, 0.0, -1.500608},
{0.893533, 0.0, -1.857415},
{0.885355, 0.0, -1.693145},
{0.877597, 0.0, -2.357478},
},
{
{0.88125, 0.0, -1.295101},
{0.891225, 0.0, -0.626773},
{0.8598

In [14]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [15]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_mean = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1, -1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.7,1)
            g_mean[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [16]:
print(f"mean_pt_params[{N_files}][{N_centr}][{N_layers}]="+"{")
for fit_functions_file in fit_mean_functions:
    print("{")
    for fit_functions_syst in fit_functions_file:
        print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
    print("},")
print("};")

mean_pt_params[1][8][6]={
{
{-0.378918, -0.238795,-0.067333, 0.010865, 0.175379, 0.329606},
{-0.314148, -0.14139,0.021462, 0.090554, 0.230241, 0.332392},
{-0.074972, -0.055868,-0.027218, -0.007548, 0.043854, 0.094371},
{-0.083594, -0.026035,0.02077, 0.042007, 0.068652, 0.079574},
{-0.004851, -0.001887,0.015061, 0.021515, 0.028418, 0.030544},
{-0.02212, 0.012339,0.032016, 0.018273, 0.012201, -0.004462},
{0.151104, 0.291368,0.281255, 0.243634, 0.13301, -0.020501},
{-0.031134, -0.097532,-0.187306, -0.211984, -0.205887, -0.033327},
},
};


In [17]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    if not is_sigmalised: dphi.append( round(0.001*layer_bins[2*ilayer]-0.0005-0.1,5))
    else: dphi.append(round(0.2*layer_bins[2*ilayer]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.9, -0.5, -0.1, 0.1, 0.5, 0.9])
[[array('d', [-0.37891808994538484, -0.23879473639860996, -0.06733268791837858, 0.010865149400329049, 0.17537909303478416, 0.32960640103183936]), array('d', [-0.3141478305756475, -0.14138968511251646, 0.021462338413217562, 0.09055395408007917, 0.23024114432876952, 0.332392256006471]), array('d', [-0.07497181781861983, -0.05586796367709948, -0.027217727628699725, -0.0075480901376055835, 0.04385378274242088, 0.09437066354853124]), array('d', [-0.0835939327609429, -0.026034618659416222, 0.02077036879235669, 0.04200650793452046, 0.06865215055350403, 0.0795740379626091]), array('d', [-0.004850587395499439, -0.0018870656057228506, 0.015060663972526336, 0.021514674788178326, 0.028418336922856478, 0.030544474175931643]), array('d', [-0.022120036321782225, 0.012338688597313567, 0.03201569919599698, 0.01827319042814554, 0.012200580721718618, -0.004461548653847815]), array('d', [0.1511043049745895, 0.2913682917174492, 0.2812552316296112, 0.243

In [18]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_mean_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_mean_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_mean_phi[ifile][icentr][ilayer][-1].Fit("pol1","","")
            g_mean_phi[ifile][icentr][ilayer][-1].GetFunction("pol1").SetLineColor(colors[icentr])
            g_mean_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

 FCN=0.22584 FROM MIGRAD    STATUS=CONVERGED      42 CALLS          43 TOTAL
                     EDM=1.62724e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -2.81905e-02   1.22195e-02   6.59791e-06   1.38212e-03
   2  p1           3.98614e-01   2.06019e-02   1.11239e-05   3.51150e-04
 FCN=3.02256 FROM MIGRAD    STATUS=CONVERGED      33 CALLS          34 TOTAL
                     EDM=9.76501e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.69889e-02   1.14708e-02   1.12083e-05  -3.57133e-02
   2  p1           3.63912e-01   1.94031e-02   1.89591e-05  -7.24781e-03
 FCN=0.99518 FROM MIGRAD    STATUS=CONVERGED      33 CALLS          34 TOTAL
                     EDM=5.99

In [19]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].Eval(pt_point)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
            dynamic_dphi_err_icentr.append((fit_functions[ifile][icentr][ilayer].GetParError(0)*2)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [1.0225781865167216, 1.0209093425864855, 1.0057259135060266, 0.9942740864939736, 1.0027120332303685, 1.0697121871085424]), array('d', [1.0565649069279794, 0.9874023942348391, 0.994996449161193, 1.0050035508388069, 1.028566280468101, 1.0140661796498252]), array('d', [1.0175221756477066, 1.0334227390676707, 1.0088410522186304, 0.9911589477813695, 0.9706429299539447, 0.9705421413187411]), array('d', [0.9837085998503854, 0.9722687461625173, 0.9921371209925928, 1.0078628790074073, 1.0288716717397655, 4.078044627130187]), array('d', [1.0317835804671553, 0.9908407963767619, 0.9995798205943829, 1.0004201794056171, 0.9912640297000265, 0.9825782806737982]), array('d', [1.0037690997425355, 1.0151314315040503, 0.9823746279639334, 1.0176253720360666, 1.0244944988677953, 1.0437445366866955]), array('d', [0.9696001621985971, 0.9604560784262796, 0.9864226771423783, 1.013577322857622, 1.0571521558753787, 1.0442659403918]), array('d', [1.0200006766772691, 1.0452296610339034, 1.0061679922221

In [20]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_sigma_phi[ifile][icentr][ilayer][-1].Fit("pol2","","",)
            g_sigma_phi[ifile][icentr][ilayer][-1].GetFunction("pol2").SetLineColor(colors[icentr])
            g_sigma_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

 FCN=0.121148 FROM MIGRAD    STATUS=CONVERGED     110 CALLS         111 TOTAL
                     EDM=5.6806e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.01342e+00   2.84495e-02   4.83236e-07  -3.23449e+00
   2  p1           2.09095e-02   6.05561e-02   1.89191e-07   2.91074e+00
   3  p2           3.45389e-02   6.43016e-02   2.10215e-07  -2.61994e+00
 FCN=0.179403 FROM MIGRAD    STATUS=CONVERGED      66 CALLS          67 TOTAL
                     EDM=1.03971e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.00559e+00   6.74496e-03   4.79500e-07   3.40320e-01
   2  p1          -8.69272e-03   6.98705e-02   3.67030e-06   2.66818e-02
   3  p2           2.81713e-02  

In [21]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_centr}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fits in fit_functions_ifile:
        print("{"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("},")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][3]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(0),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(1),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("},")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][2]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(0),6)},{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("},")
print("};")



const float phi_sigma_pt_params[1][8][3]={
{
{0.523768, 0.40027, -0.31897},
{0.872888, 0.0, -1.215096},
{0.867479, 0.0, -1.348911},
{0.448231, 0.431465, -0.16251},
{0.893157, 0.0, -1.679012},
{0.876636, 1.7e-05, 4.154985},
{0.167704, 0.801776, -0.332122},
{0.175557, 0.774892, -0.257894},
},
};
const float phi_sigma_phi_params[1][8][3]={
{
{1.01342,0.020909,0.034539},
{1.005585,-0.008693,0.028171},
{1.003258,-0.055049,0.010509},
{1.028104,0.421635,0.619856},
{0.997494,-0.023925,0.008174},
{1.018419,0.001322,-0.011076},
{1.004658,0.08218,0.045176},
{1.001779,-0.085376,0.00086},
},
};
const float phi_mean_phi_params[1][8][2]={
{
{-0.02819,0.398614},
{0.036989,0.363912},
{-0.004741,0.096696},
{0.017052,0.092842},
{0.01482,0.022244},
{0.008201,0.007693},
{0.189394,-0.121313},
{-0.13555,-0.043248},
},
};
